# Desafío - Clasificación desde la econometría

## Descripción
En esta sesión trabajaremos el dataset south african heart, el cual contiene las siguientes
variables:
<li><code>sbp</code>: Presión Sanguínea Sistólica.</li>
<li><code>tobacco</code>: Promedio tabaco consumido por día.</li>
<li><code>ldl</code>: Lipoproteína de baja densidad.</li>
<li><code>adiposity</code>: Adiposidad.</li>
<li><code>famhist</code>: Antecedentes familiares de enfermedades cardiácas. (Binaria)</li>
<li><code>types</code>: Personalidad tipo A</li>
<li><code>obesity</code>: Obesidad.</li>
<li><code>alcohol</code>: Consumo actual de alcohol.</li>
<li><code>age</code>: edad.</li>
<li><code>chd</code>: Enfermedad coronaria. (dummy)</li>


## Desafío 1: Preparar el ambiente de trabajo
<li>Cargue las librerías básicas para importación y manipulación de datos (numpy,
pandas), gráficos (matplotlib y seaborn) y de modelación econométrica
(statsmodels).</li>
<li>Importe el archivo southafricanheart.csv que se encuentra dentro del material de
apoyo.</li>
<li>Realice una descripción del set importado mostrando:
<ul>
<li>lista con los nombres de variables importadas</li>
<li>un análisis descriptivo mediante <code>.describe()</code></li>
<li>distribución de categorías para las variables <code>famhist</code> y <code>chd</code>.</li></ul></li>


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import scipy.stats as stats

df = pd.read_csv('southafricanheart.csv').drop(columns=['Unnamed: 0'])

print(df.describe())
print("="*24)
print(df['famhist'].value_counts())
print("="*24)
print(df['chd'].value_counts())



              sbp     tobacco         ldl   adiposity       typea     obesity  \
count  462.000000  462.000000  462.000000  462.000000  462.000000  462.000000   
mean   138.326840    3.635649    4.740325   25.406732   53.103896   26.044113   
std     20.496317    4.593024    2.070909    7.780699    9.817534    4.213680   
min    101.000000    0.000000    0.980000    6.740000   13.000000   14.700000   
25%    124.000000    0.052500    3.282500   19.775000   47.000000   22.985000   
50%    134.000000    2.000000    4.340000   26.115000   53.000000   25.805000   
75%    148.000000    5.500000    5.790000   31.227500   60.000000   28.497500   
max    218.000000   31.200000   15.330000   42.490000   78.000000   46.580000   

          alcohol         age         chd  
count  462.000000  462.000000  462.000000  
mean    17.044394   42.816017    0.346320  
std     24.481059   14.608956    0.476313  
min      0.000000   15.000000    0.000000  
25%      0.510000   31.000000    0.000000  
50%   

## Desafío 2
A continuación se presenta el siguiente modelo a estimar:

$$
    log(\frac{Pr(chd=1)}{1-Pr(chd = 1)}) = \beta_0 + \beta_1 \cdot famhist
$$
Para ello ejecute los siguientes pasos:
<ol>
<li>Recodifique <code>famhist</code> a dummy, asignando 1 a la categoría minoritaria.</li>
<li>Utilice <code>smf.logit</code> para estimar el modelo.</li>
<li>Implemente una función <code>inverse_logit</code> que realice el mapeo de log-odds a
probabilidad.</li>
<li>Con el modelo estimado, responda lo siguiente:
<ul>
<li>¿Cuál es la probabilidad de un individuo con antecedentes familiares de tener
una enfermedad coronaria?</li>
<li>¿Cuál es la probabilidad de un individuo sin antecedentes familiares de tener
una enfermedad coronaria?</li>
<li>¿Cuál es la diferencia en la probabilidad entre un individuo con antecedentes
y otro sin antecedentes?</li>
<li>Replique el modelo con <code>smf.ols</code> y comente las similitudes entre los
coeficientes estimados.
</br><b>Tip:</b> Utilice β/4</li>
</ul></li>
<ol>


In [2]:
def concise_summary(mod, print_fit=True):
    #guardamos los parámetros asociados a estadísticas de ajuste
    fit = pd.DataFrame({'Statistics': mod.summary2().tables[0][2][2:],'Value': mod.summary2().tables[0][3][2:]})
    # guardamos los parámetros estimados por cada regresor.
    estimates = pd.DataFrame(mod.summary2().tables[1].loc[:, 'Coef.':'Std.Err.'])
    # imprimir fit es opcional
    if print_fit is True:
        print("\nGoodness of Fit statistics\n", fit)
        print("\nPoint Estimates\n\n", estimates)
    return (fit, estimates)
df['famhist'] = np.where(df['famhist'] == 'Present', 1, 0)

In [3]:
model = smf.logit('chd ~ famhist', df).fit()
fit, estimates = concise_summary(model, True)

Optimization terminated successfully.
         Current function value: 0.608111
         Iterations 5

Goodness of Fit statistics
         Statistics       Value
2             BIC:    574.1655
3  Log-Likelihood:     -280.95
4         LL-Null:     -298.05
5     LLR p-value:  4.9371e-09
6           Scale:      1.0000
7                             

Point Estimates

               Coef.  Std.Err.
Intercept -1.168993  0.143106
famhist    1.168993  0.203255


In [4]:
def inverse_logit(x):
    return round(1 / (1+np.exp(-x)), 2)
print("Para la regresión Logística")
# Probabilidad cuando tiene historial familiar
famhistProb = inverse_logit(estimates['Coef.'][0] + estimates['Coef.'][1])
print("Probabilidad cuando se tiene historial familiar: {}".format(famhistProb))
# Probabilidad cuando no tiene historial familiar
noFamhistProb = inverse_logit(estimates['Coef.'][0])
print("Probabilidad cuando no se tiene historial familiar: {}".format(noFamhistProb))
# Diferencia de probabilidad
print("Diferencia de probabilidad: {}".format(famhistProb - noFamhistProb))

Para la regresión Logística
Probabilidad cuando se tiene historial familiar: 0.5
Probabilidad cuando no se tiene historial familiar: 0.24
Diferencia de probabilidad: 0.26


In [5]:
print("Para la regresión Lineal")
ols_model = smf.ols("chd ~ famhist", df).fit()
_, estimates_ols = concise_summary(ols_model, True)
# Probabilidad lineal cuando tiene historial familiar
famhistProb = estimates_ols['Coef.'][0] + estimates_ols['Coef.'][1]
print("Probabilidad lineal cuando se tiene historial familiar: {}".format(famhistProb))
# Probabilidad lineal cuando no tiene historial familiar
noFamhistProb = estimates_ols['Coef.'][0]
print("Probabilidad lineal cuando no se tiene historial familiar: {}".format(noFamhistProb))
# Diferencia de probabilidad lineal
print("Beta cuartos de la regresión logística: {}/4 = {} ~ {}.".format(estimates['Coef.'][1], round(estimates['Coef.'][1]/4, 5), round(estimates_ols['Coef.'][1], 5)))

Para la regresión Lineal

Goodness of Fit statistics
             Statistics     Value
2                 BIC:  601.4437
3      Log-Likelihood:   -294.59
4         F-statistic:     36.86
5  Prob (F-statistic):  2.66e-09
6               Scale:   0.21050

Point Estimates

               Coef.  Std.Err.
Intercept  0.237037  0.027922
famhist    0.262963  0.043313
Probabilidad lineal cuando se tiene historial familiar: 0.4999999999999999
Probabilidad lineal cuando no se tiene historial familiar: 0.23703703703703696
Beta cuartos de la regresión logística: 1.1689930854299089/4 = 0.29225 ~ 0.26296.


Notamos que el modelo logístico y el lineal tienen una predicción parecida para cuando se tiene historial familiar, no asi el caso en el que no.

## Desafío 3: Estimación completa
Implemente un modelo con la siguiente forma:
$$
    log(\frac{Pr(chd=1)}{1-Pr(chd = 1)}) = \beta_0 + \sum_{j=1}^{N}\beta_j \cdot X
$$
<ul>
<li>Depure el modelo manteniendo las variables con significancia estadística al 95%.</li>
<li>Compare los estadísticos de bondad de ajuste entre ambos.</li>
<li>Reporte de forma sucinta el efecto de las variables en el log-odds de tener una
enfermedad coronaria.</li>
</ul>


In [6]:
full_model = smf.logit('chd ~ famhist + sbp + tobacco + ldl + adiposity + typea + obesity + alcohol + age', df).fit()
concise_summary(full_model, True)
print(full_model.pvalues)

Optimization terminated successfully.
         Current function value: 0.510974
         Iterations 6

Goodness of Fit statistics
         Statistics       Value
2             BIC:    533.4957
3  Log-Likelihood:     -236.07
4         LL-Null:     -298.05
5     LLR p-value:  2.0548e-22
6           Scale:      1.0000
7                             

Point Estimates

               Coef.  Std.Err.
Intercept -6.150721  1.308260
famhist    0.925370  0.227894
sbp        0.006504  0.005730
tobacco    0.079376  0.026603
ldl        0.173924  0.059662
adiposity  0.018587  0.029289
typea      0.039595  0.012320
obesity   -0.062910  0.044248
alcohol    0.000122  0.004483
age        0.045225  0.012130
Intercept    0.000003
famhist      0.000049
sbp          0.256374
tobacco      0.002847
ldl          0.003555
adiposity    0.525700
typea        0.001310
obesity      0.155095
alcohol      0.978350
age          0.000193
dtype: float64


In [7]:
# se eliminan aquellas con p-value > 0.05
depurated_model = smf.logit('chd ~ famhist + tobacco + ldl + typea + age', df).fit()
fit_depurated, estimates_depurated = concise_summary(depurated_model, True)

Optimization terminated successfully.
         Current function value: 0.514811
         Iterations 6

Goodness of Fit statistics
         Statistics       Value
2             BIC:    512.4990
3  Log-Likelihood:     -237.84
4         LL-Null:     -298.05
5     LLR p-value:  2.5537e-24
6           Scale:      1.0000
7                             

Point Estimates

               Coef.  Std.Err.
Intercept -6.446445  0.920872
famhist    0.908175  0.225758
tobacco    0.080375  0.025880
ldl        0.161992  0.054969
typea      0.037115  0.012167
age        0.050460  0.010206


Se observa que el nuevo modelo tiene estadísticos de bondad muy parecidos al modelo original, siendo lo mas notable la disminución de <code>LLR p-value</code> en un par de ordenes de magnitud.

## Desafío 4: Estimación de perfiles
A partir del modelo depurado, genere las estimaciones en log-odds y posteriormente
transfórmelas a probabilidades con <code>inverse_logit</code>. Los perfiles a estimar son los
siguientes:
<ul>
<li>La probabilidad de tener una enfermedad coronaria para un individuo con
características similares a la muestra.</li>
<li>La probabilidad de tener una enfermedad coronaria para un individuo con altos
niveles de lipoproteína de baja densidad, <b>manteniendo todas las demás
características constantes</b>.</li>
<li>La probabilidad de tener una enfermedad coronaria para un individuo con bajos
niveles de lipoproteína de baja densidad,<b> manteniendo todas las demás
características constantes</b>.</li>
</ul>

In [8]:
# conseguimos una persona cuyos valores sean igual a los promedios de cada columna respectivamente
def getLogOdd(observation, columns, estimates):
    logOdd = estimates['Coef.']['Intercept']
    for i in columns:
        logOdd += estimates['Coef.'][i]*observation[i]
    return logOdd
meanPerson = df.mean()

meanLogOdd = getLogOdd(meanPerson, ['famhist', 'tobacco', 'ldl', 'typea', 'age'], estimates_depurated)
print("Una persona promedio tiene log odd {} y probabilidad {} de tener una enfermedad coronaria".format(round(meanLogOdd, 5), inverse_logit(meanLogOdd)))


Una persona promedio tiene log odd -0.87744 y probabilidad 0.29 de tener una enfermedad coronaria


In [9]:
# Para las siguientes consideraremos que todos las columnas son iguales al promedio previo, con la diferencia de ldl
ldlStats = df['ldl'].describe()
highLDL = meanPerson.copy()
highLDL['ldl'] = ldlStats['max']
minLDL = meanPerson.copy()
minLDL['ldl'] = ldlStats['min']

minLogOdd = getLogOdd(minLDL, ['famhist', 'tobacco', 'ldl', 'typea', 'age'], estimates_depurated)
print("Una persona con bajo nivel de LDL tiene log odd {} y probabilidad {} de tener una enfermedad coronaria".format(round(minLogOdd, 5), inverse_logit(minLogOdd)))
maxLogOdd = getLogOdd(highLDL, ['famhist', 'tobacco', 'ldl', 'typea', 'age'], estimates_depurated)
print("Una persona con alto nivel de LDL tiene log odd {} y probabilidad {} de tener una enfermedad coronaria".format(round(maxLogOdd, 5), inverse_logit(maxLogOdd)))

Una persona con bajo nivel de LDL tiene log odd -1.48658 y probabilidad 0.18 de tener una enfermedad coronaria
Una persona con alto nivel de LDL tiene log odd 0.838 y probabilidad 0.7 de tener una enfermedad coronaria
